# Homework 4: CKY Algorithm and Dependency Parsing

### 1: CKY Algorithm (30 points)

In this section, you will implement the CKY algorithm for an unweighted CFG. See the starter code [cky.py](http://people.cs.umass.edu/~brenocon/inlp2017/hw4/cky.py).

##### Question 1.1 (8 points)
Implement the acceptance version of CKY as ``cky_acceptance()``, which returns True if there is a ``S`` covering the entire sentence. Does it return True or False for the following sentences? Please ``pprint()`` the chart cells for each case as well. 
* the the
* the table attacked a dog
* the cat

Hint: A simple way to implement the chart cells is by maintaining a list of nonterminals at the span. This list represents all possible nonterminals over that span. 

Hint: ``pprint()``ing the CKY chart cells may be useful for debugging.

Hint: Python dictionaries allow tuples as keys. For example, ``d={}; d[(3,4)] = []``. A slight shortcut is that ``d[3,4]`` means the same thing as ``d[(3,4)]``.

In [2]:
import cky
from pprint import pprint
print "Grammar rules in tuple form:"
pprint(cky.grammar_rules)
print "Rule parents indexed by children:"
pprint(cky.possible_parents_for_children)

Grammar rules in tuple form:
[('S', ('NP', 'VP')),
 ('NP', ('Det', 'Noun')),
 ('VP', ('Verb', 'NP')),
 ('PP', ('Prep', 'NP')),
 ('NP', ('NP', 'PP')),
 ('VP', ('VP', 'PP'))]
Rule parents indexed by children:
{('Det', 'Noun'): ['NP'],
 ('NP', 'PP'): ['NP'],
 ('NP', 'VP'): ['S'],
 ('Prep', 'NP'): ['PP'],
 ('VP', 'PP'): ['VP'],
 ('Verb', 'NP'): ['VP']}


In [3]:
#Print the result here
import cky;reload(cky)
print "the the"
print cky.cky_acceptance(["the", "the"])
print "the table attacked a dog"
print cky.cky_acceptance(["the", "table", "attacked", "a", "dog"])
print "the cat"
print cky.cky_acceptance(["the", "cat"])

the the
False
the table attacked a dog
True
the cat
False


##### Question 1.2 (15 points)
Implement the parsing version of CKY, which returns one of the legal parses for the sentence (and returns None if there are none). If there are multiple real parses, we don’t care which one you print. Implement this as `cky_parse()`. You probably want to start by copying your `cky_acceptance()` answer and modifying it. Have it return the parse in the following format, using nested lists to represent the tree (this is a simple Python variant of the Lisp-style S-expression that’s usually used for this.)

```
['S',
        [['NP', [['Det', 'the'], ['Noun', 'cat']]],
         ['VP', [['Verb', 'attacked'], 
                 ['NP', [['Det', 'the'], ['Noun', 'food']]]]]]]
                 ```

Print out the parses for the following sentences.  
* the cat saw a dog
* the cat saw a dog in a table
* the cat with a table attacked the food  

Hint: In the chart cells, you will now have to store backpointers as well. One way to do it is to store a list of tuples, each of which is  ``(nonterminal, splitpoint, leftchild nonterm, rightchild nonterm)``. For example, if the state ``('NP', 3, 'Det', 'Noun')`` is in the cell for span (2,4), that means this is a chart state of symbol NP, which came from a ``Det`` at position (2,3) and a Noun at position (3,4).

Hint: It may be useful to use a recursive function for the backtrace.

In [6]:
# Output the results for each sentence.
#TODO: Print out the parse tree for each of the three sentence

pprint(cky.cky_parse(['the','cat','saw','a','dog']))
print "*" * 50

pprint(cky.cky_parse(['the','cat','saw','a','dog','in','a','table']))
print "*" * 50

pprint( cky.cky_parse(['the','cat','with','a','table','attacked','the','food']) )
print "*" * 50

['S',
 [['NP', [['Det', 'the'], ['Noun', 'cat']]],
  ['VP', [['Verb', 'saw'], ['NP', [['Det', 'a'], ['Noun', 'dog']]]]]]]
**************************************************
['S',
 [['NP', [['Det', 'the'], ['Noun', 'cat']]],
  ['VP',
   [['Verb', 'saw'],
    ['NP',
     [['NP', [['Det', 'a'], ['Noun', 'dog']]],
      ['PP', [['Prep', 'in'], ['NP', [['Det', 'a'], ['Noun', 'table']]]]]]]]]]]
**************************************************
['S',
 [['NP',
   [['NP', [['Det', 'the'], ['Noun', 'cat']]],
    ['PP', [['Prep', 'with'], ['NP', [['Det', 'a'], ['Noun', 'table']]]]]]],
  ['VP', [['Verb', 'attacked'], ['NP', [['Det', 'the'], ['Noun', 'food']]]]]]]
**************************************************


##### Question 1.3 (7 points)
Revise the grammar as follows.

* Add four words to the lexicon: two verbs “attack” and “attacks”, and the nouns “cats” and “dogs”.
* Revise the rules to enforce subject-verb agreement on number.

The new grammar should accept and reject the following sentences. Please run your parser on these sentences and report the parse trees for the accepted ones. Also, describe how you changed the grammar, and why.

ACCEPT: ``the cat attacks the dog``   
REJECT: ``the cat attack the dog``  
ACCEPT: ``the cats attack the dog``  
REJECT: ``the cat with the food on a dog attack the dog``

Hint: you will need to introduce new nonterminal symbols, and modify the currently existing ones.

### Revise:
The new rule and lexicon is following. Classify nouns and verbs in single and plural form and make rules accordingly. The new parser confirms its correctness. None result means it's invalid.

grammar_text = """
S -> NP VP
S -> NP_s VP_s
NP -> Det Noun
VP -> Verb NP
VP -> Verb NP_s
VP_s -> Verb_s NP
VP_s -> Verb_s NP_s
NP_s ->Det Noun_s
PP -> Prep NP
NP -> NP PP
VP -> VP PP
"""

lexicon = {
    'Noun': set(['cats', 'dogs', 'table', 'food']),
    'Noun_s' : set(['cat','dog']),
    'Verb': set(['attacked', 'attack','saw', 'loved', 'hated']),
    'Verb_s':set(['attacks']),
    'Prep': set(['in', 'of', 'on', 'with']),
    'Det': set(['the', 'a']),
}

In [24]:
# Output the results for each sentence.
#TODO: Print out the parse tree for each of the four sentence
reload(cky)
pprint( cky.cky_parse(['the','cat','attacks','the','dog']) )
print "*" * 50
#TODO: Print out the parse tree for each of the four sentence
pprint( cky.cky_parse(['the','cat','attack','the','dog']) )
print "*" * 50
#TODO: Print out the parse tree for each of the four sentence
pprint( cky.cky_parse(['the','cats','attack','the','dog']) )
print "*" * 50
#TODO: Print out the parse tree for each of the four sentence
pprint( cky.cky_parse(['the','cat','with','the','food','on','a','dog','attack','the','dog']) )
print "*" * 50

['S',
 [['NP_s', [['Det', 'the'], ['Noun_s', 'cat']]],
  ['VP_s',
   [['Verb_s', 'attacks'], ['NP_s', [['Det', 'the'], ['Noun_s', 'dog']]]]]]]
**************************************************
None
**************************************************
['S',
 [['NP', [['Det', 'the'], ['Noun', 'cats']]],
  ['VP', [['Verb', 'attack'], ['NP_s', [['Det', 'the'], ['Noun_s', 'dog']]]]]]]
**************************************************
None
**************************************************


### 2: Weighted CKY Algorithm (40 points)
In this section you will implement the weighted CKY Algorithm for a Probabilistic CFG. You will have to make modifications to the existing algorithm to account for the probabilities and your parse function should output the most probable parse tree. 
Please write all your code in ``weighted_cky.py`` file for this section. 

##### Question 2.1 (7 points)
The CKY Algorithm requires the CFG to be in Chomsky Normal Form. Convert the following CFG into Chomsky Normal Form. (For the sake of uniformity, replace the leftmost pairs of non-terminals with new non-terminal)

S -> Aux NP VP   
S -> VP  
VP -> Verb NP  
VP -> VP PP  
Verb -> book  
Aux -> does  

### Answer here
    S -> Aux_NP VP 
    S -> VP 
    Aux_NP -> Aux NP 
    VP -> Verb NP 
    VP -> VP PP 
    Verb -> book 
    Aux -> does 


##### Question 2.2 (8 points)
We will now implement a weighted CYK algorithm to parse a sentence and return the most probable parse tree. 
The grammar is defined in ``pcfg_grammar_original.txt``. As you can notice, some of the rules are not in CNF. 
Modify the ``pcfg_grammar_modified.txt`` file such that all the rules are in Chomsky Normal Form.
(For the sake of uniformity, replace the leftmost pairs of non-terminals with new non-terminal)

Note: When transforming the grammar to CNF, must set production probabilities to preserve the probability of derivations.

###### Question 2.3 (5 points)
Explain briefly what are the changes you made to convert the grammar into CNF Form. Why did you make these changes?

### Answer:
1. conbine `Aux` and `NP`in `S -> Aux NP VP` as `Aux_NP`, a new non-termnal. add a new rule `Aux_NP -> Aux NP 1`
This way the rules are binarized 
2. make 'Houston' to 'houston', 'I' to 'i' and 'NWA' to 'nwa' to differentiate the unary rules and lexicon terms

##### Question 2.4 (8 points)
Complete the ``populate_grammar_rules()`` function in the ``weighted_cky.py`` script. This function will have to read in the grammar rules from ``pcfg_grammar_modified.txt`` file and populate the `grammar_rules` and `lexicon` data structure. Additionally you would need to store the probability mapping in a suitable data structure. 

Hint: You can modify the starter code provided in cky.py for this task. 

In [4]:
import weighted_cky as wcky;reload(wcky)

wcky.populate_grammar_rules()

Grammar rules in tuple form:
[('S', ('NP', 'VP')),
 ('S', ('Aux_NP', 'VP')),
 ('Aux_NP', ('Aux', 'NP')),
 ('S', 'VP'),
 ('NP', 'Pronoun'),
 ('NP', 'Proper-Noun'),
 ('NP', ('Det', 'Nominal')),
 ('Nominal', 'Noun'),
 ('Nominal', ('Nominal', 'Noun')),
 ('Nominal', ('Nominal', 'PP')),
 ('VP', 'Verb'),
 ('VP', ('Verb', 'NP')),
 ('VP', ('VP', 'PP')),
 ('PP', ('Prep', 'NP'))]
probabilities
{('Aux', 'does'): '1.0',
 ('Aux_NP', ('Aux', 'NP')): '1',
 ('Det', 'a'): '0.2',
 ('Det', 'that'): '0.1',
 ('Det', 'the'): '0.6',
 ('Det', 'this'): '0.1',
 ('NP', 'Pronoun'): '0.2',
 ('NP', 'Proper-Noun'): '0.2',
 ('NP', ('Det', 'Nominal')): '0.6',
 ('Nominal', 'Noun'): '0.3',
 ('Nominal', ('Nominal', 'Noun')): '0.2',
 ('Nominal', ('Nominal', 'PP')): '0.5',
 ('Noun', 'book'): '0.1',
 ('Noun', 'flight'): '0.5',
 ('Noun', 'meal'): '0.2',
 ('Noun', 'money'): '0.2',
 ('PP', ('Prep', 'NP')): '1.0',
 ('Prep', 'from'): '0.25',
 ('Prep', 'near'): '0.2',
 ('Prep', 'on'): '0.1',
 ('Prep', 'through'): '0.2',
 ('Prep', 

##### Question 2.5 (12 points)
Implement the weighted parsing version of CKY, which returns the most probable legal parse for the sentence (and returns None if there are none). If there are multiple real parses, this function will always return the most probable parse i.e the one with maximum probability. 
Complete the ``pcky_parse()``.
Print the parse tree and the probabilities for the following sentences:
* book the flight through Houston
* include this book
* the the

Hint: You can use the code in `cky_parse()` and modify it to accomodate probabilities and compute the most probable parse.   
Note: The topmost cell should contain rules associated with the `S` non terminal, if any.

In [9]:
import weighted_cky as wcky;reload(wcky)
from pprint import pprint
# Output the results for each sentence.
#TODO: Print out the parse tree for each of the three sentence
pprint( wcky.pcky_parse(['the','the']) )
print "*" * 50
pprint( wcky.pcky_parse(['include','this','book']) )
print "*" * 50
pprint(wcky.pcky_parse(['book', 'the', 'flight', 'through', 'Houston']))

None
**************************************************
[('S', 1.8e-05),
 [([('VP', 0.00018),
    [(['Verb', 'include'], 0.2),
     ([('NP', 0.0018),
       [(['Det', 'this'], 0.1), (['Nominal', 'book'], 0.03)]],
      0.0018)]],
   0.00018)]]
**************************************************
[('S', 2.1600000000000007e-05),
 [([('VP', 0.00021600000000000005),
    [([('VP', 0.0135),
       [(['Verb', 'book'], 0.5),
        ([('NP', 0.054),
          [(['Det', 'the'], 0.6), (['Nominal', 'flight'], 0.15)]],
         0.054)]],
      0.0135),
     ([('PP', 0.03200000000000001),
       [(['Prep', 'through'], 0.2),
        (['NP', 'Houston'], 0.16000000000000003)]],
      0.03200000000000001)]],
   0.00021600000000000005)]]


### 3: Dependency parser output (30 points)

You will conduct manual error analysis of [CoreNLP](https://stanfordnlp.github.io/CoreNLP/)'s dependency parser.

Create an English sentence where the parser makes an error, and you know what the correct analysis ought to be, according to the Universal Dependencies grammatical standard.  You will want to play around with different sentences, look at their output, and check against the Universal Dependencies annotation standard.  The current version of CoreNLP outputs according to the "UD version 1" standard, so please use this page:
 * [UD v1 homepage](http://universaldependencies.org/docsv1/)
 * and in particular, the [UD v1 dependency relations list](http://universaldependencies.org/docsv1/u/dep/index.html)

For quickly looking at things, their [online demo](http://corenlp.run/) may be useful.

However, for this assignment, you need to run the parser to output in "conllu" format, which is human readable.  You need to download and run the parser for this.  (It requires Java.) Use version 3.8.0 (it should be the current version). You can it working in interactive mode so you can just type sentences into it on the terminal like this:

```
./corenlp.sh -annotators tokenize,ssplit,pos,lemma,depparse -outputFormat conllu 
[...]
Entering interactive shell. Type q RETURN or EOF to quit.
NLP> 
```

For example then you can type
```
NLP> I saw a cat.
1       I       I       _       PRP     _       2       nsubj   _       _
2       saw     see     _       VBD     _       0       root    _       _
3       a       a       _       DT      _       4       det     _       _
4       cat     cat     _       NN      _       2       dobj    _       _
5       .       .       _       .       _       2       punct   _       _
```

You can also use the `-inputFile` flag if you'd rather give it a whole file at once.

As you can see in the parser documentation, the 7th and 8th columns describe the dependency edge for the word's parent (a.k.a governor): it has the index of its parent, and the edge label (a.k.a. the relation).  For example, this parse contains the dependency edge *nsubj(saw:2, I:1)* meaning that "I" is the subject of "saw".

**Question 3.1:** Once you've decided your sentence, please put the conllu-formatted parser output below in the markdown triple-quoted area.  Please be very careful where it goes since we will use a script to pull it out.
    
PARSE GOES BELOW HERE
```
1	Do	do	_	VB	_	8	csubj	_	_
2	show	show	_	NN	_	4	compound	_	_
3	1	1	_	CD	_	4	nummod	_	_
4	example	example	_	NN	_	1	dobj	_	_
5	of	of	_	IN	_	7	case	_	_
6	that	that	_	DT	_	7	det	_	_
7	NP	np	_	NN	_	4	nmod	_	_
8	equals	equal	_	VBZ	_	0	root	_	_
9	P	p	_	NN	_	8	dobj	_	_

```
PARSE GOES ABOVE HERE

**Question 3.2:** Please describe the error you found.  Also give a citation and link to the relevant part of the UD documentation describing one of the relations that the parser predicted in error, or did something wrong with.               

**`Answer` :**  

`'Do'` has the dependency edge `'csubj'`(`'Do'`:1, `'equals'`:8), meaning the `'Do'` clause is the clause subject of `'equals'`.
`'show'` has dependency edge `'compound'`(`'show'`:2,`'example'`:4), which is clearly incorrect. 
The CoreNLP could be incorrect in parsing the subjectless sentences(imperatives in this case) with ambiguities(`'show'` can be both verb and noun).  

For citation and link, see section 3.6:
https://catalog.ldc.upenn.edu/docs/LDC99T42/prsguid2.pdf


**Question 3.3:** Please give correct that error in the parse .  Put your corrected parse, again in that conllu textual format, below.  You should take a copy of the output and manually change some of the 7th/8th dependency edge columns.

PARSE GOES BELOW HERE
```
1	Do	    do	    _	VB	_	2	aux	_	_
2	show	show	_	VB	_	0	root_	_
3	1	    1	    _	CD	_	4	num	_	_
4	example	example	_	NN	_	1	dobj	_	_
5	of	    of	    _	IN	_	6	case	_	_
6	that	that	_	IN	_	8	mark	_	_
7	NP	    np	    _	NN	_	8	nsubj	_	_
8	equals	equal	_	VBZ	_	0	ccomp	_	_
9	P	    p	    _	NN	_	8	dobj	_	_

```
PARSE GOES ABOVE HERE

**Question 3.4:** Please describe your correction and why it solves the error.   

**Answer : **

I made *'Do'* as *'aux'* with dependency edge 2, *'show'* as root, *'that'* as *'mark'* with dependency edge 8. *'NP'* as *'nsubj'* with dependency edge 8.   
This way, *'do'* is parsed as an aux verb of *'show'*, which is the verb, as it should be as we know it. *'that'* is corrected as a mark of object clause, which is what we know of.And the others simply follow


